Last updated: July 5, 2023

Last run: July 5, 2023

**Outlet filtering and identification of top outlets**

## Unreviewed science in the news: The evolution of preprint media coverage from 2014-2021

Juan Pablo Alperin

**Related Publication:**
Fleerackers, A., Shores, K., Chtena, N. & Alperin, J.P. (2023). Unreviewed science in the news: The evolution of preprint media coverage from 2014-2021. *bioarxiv*. 

**Related Dataset:**
Alperin, J. P., Fleerackers, A., & Shores, K. (2023). Data for: Unreviewed science in the news [Data set]. Harvard Dataverse. https://doi.org/10.7910/DVN/ZHQUFD

Note: the unfiltered data used by this script is not available in the accompanying dataset because it includes unfiltered data from the Web of Science that cannot be shared due to contractual obligations. The original data was sourced from the OST, housed at UQAM.

In [1]:
import pandas as pd

### Altmetric news mentions of items indexed in Web of Science 

`news_of_wos_with_doi_mention_details.csv` was exported from the database housed at the OST (UQAM)

```
SELECT 
      n.Altmetric_ID, n.[Order], alt.DOI, alt.Arxiv_ID, alt.First_Seen_On, alt.PubDate, alt.Title as alt_title, a.Annee_Bibliographique as wos_year, a.Titre as wos_title, n.Author_Name, n.Author_Url, n.Posted_On, n.Title, n.Url as moreover_url
  FROM [WoS].[pex].[Article]  a INNER JOIN [WoS].[dbo].[Identifier] i ON (a.OST_BK = i.OST_BK AND i.Type = 'doi')
  JOIN [Altmetrics_2021Jun3].[dbo].[Document] alt ON (i.Identifier = alt.DOI )
  JOIN [Altmetrics_2021Jun3].[dbo].[News] n ON (alt.Altmetric_ID = n.Altmetric_ID)
  WHERE n.Posted_On >= '2014'
```

In [2]:
input_file = 'data/news_of_wos_with_doi_mention_details.csv'
columns = ['Altmetric_ID', 'Order', 'DOI', 'Arxiv_ID', 'First_Seen_On', 'PubDate', 'alt_title', 'wos_year', 'wos_title', 'Author_Name', 'Author_Url', 'Posted_On', 'Title', 'moreover_url']

In [3]:
df = pd.read_csv(input_file, sep=';', names=columns)
df.columns = [x.lower() for x in df.columns]
df.rename(columns = {'author_name': 'outlet', 'author_url': 'outlet_url'}, inplace=True)
df['posted_on'] = pd.to_datetime(df.posted_on)
df['posted_on_year'] = df.posted_on.map(lambda x: x.year)

/Users/juan/.pyenv/versions/myenv3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
"Number of WoS mentions: {:,}".format(df.shape[0])

'Number of WoS mentions: 6,053,470'

In [5]:
df2 = df.groupby(['outlet', 'posted_on_year']).size().unstack()

# Identify outlets that have at least 100 stories EVERY year between 2014 and 2020 (not 2021)
df2 = df2[df2[list(range(2014, 2021))].apply(lambda x: x >= 100).all(axis=1)]

print("Number of unique outlets with at least 100 stories each year (prior to exclusions): %s" % len(df2.index.unique()))

Number of unique outlets with at least 100 stories each year (prior to exclusions): 128


In [6]:
exclude = pd.read_csv('data/excluded_outlets.csv', encoding='utf8')

print('Mentions prior to exclusion: {:,}'.format(df.shape[0]))
print('Excluding from {} outlets:'.format(exclude.shape[0]))
t = 0
for o in exclude.Outlet:
    t += df[df['outlet'] == o].shape[0]
    print('{}: {:,}'.format(o, df[df['outlet'] == o].shape[0]))
        
print()
print('Total exclusions: {:,}'.format(t))
del t
df = df[~df.outlet.isin(exclude.Outlet)]
print('Remaining mentions: {:,}'.format(df.shape[0]))

Mentions prior to exclusion: 6,053,470
Excluding from 34 outlets:
Pansci: 1,542
RT Network: 7,280
Photonics.com: 2,455
Helsingin Sanomat: 3,929
Futura-Sciences: 8,234
Sciences et Avenir: 7,355
Top Santé: 3,401
L'Express: 1,010
Der Tagesspiegel: 1,793
Deutsches Ärzteblatt: 7,994
Der Standard: 3,423
Spektrum: 13,442
SPIEGEL ONLINE: 5,764
Innovations Report: 14,208
Neue Züricher Zeitung (NZZ): 2,837
Informationsdienst Wissenschaft: 24,740
Jura Forum: 7,746
Die Welt: 4,311
Ingeniøren: 1,592
Ad Hoc News: 1,595
LABO Online: 2,575
Corriere della Sera: 1,691
Le Scienze: 5,042
Galileonet: 4,306
El País: 6,644
Agencia SINC: 7,281
NCYT - Noticias de la Ciencia y la Technologia: 5,082
MSN: 101,829
MIT Technology Review: 2,355
Bioportfolio: 38,023
Health Canal: 18,283
Mother Nature Network: 3,525
NPR: 5,073
Health24: 4,582

Total exclusions: 330,942
Remaining mentions: 5,722,528


In [7]:
df2 = df.groupby(['outlet', 'posted_on_year']).size().unstack()

# Identify outlets that have at least 100 stories EVERY year between 2014 and 2020 (not 2021)
df2 = df2[df2[list(range(2014, 2021))].apply(lambda x: x >= 100).all(axis=1)]
df2.to_csv('data/top outlets/outlets.csv')
df = df[df.outlet.isin(df2.index)]

print("Number of unique outlets with at least 100 stories each year: %s" % len(df.outlet.unique()))

Number of unique outlets with at least 100 stories each year: 94


In [8]:
df.to_csv('data/top outlets/wos_with_doi_mention_details.csv', index=False)

In [9]:
"We downloaded all of the mentions of WoS research from our {:,} outlets, resulting in {:,} mentions of {:,} distinct research outputs in {:,} distinct news stories".format(df.outlet.unique().shape[0], df.shape[0], df.altmetric_id.unique().shape[0], df[['outlet', 'moreover_url']].drop_duplicates().shape[0])


'We downloaded all of the mentions of WoS research from our 94 outlets, resulting in 1,657,202 mentions of 466,138 distinct research outputs in 1,113,270 distinct news stories'

### Altmetric news mentions of items published in preprint servers

`news_of_preprints_mention_details.csv` was exported from the database housed at the OST (UQAM)

```
  SELECT 
      n.Altmetric_ID, n.[Order], alt.DOI, alt.Arxiv_ID, j.Journal, j.Altmetric_JID, alt.First_Seen_On, alt.PubDate, alt.Title as alt_title, n.Author_Name, n.Author_Url, n.Posted_On, n.Title, n.Url as moreover_url
  FROM
  [Altmetrics_2021Jun3].[dbo].[Document] alt 
  JOIN [Altmetrics_2021Jun3].[dbo].[News] n ON (alt.Altmetric_ID = n.Altmetric_ID)
  LEFT JOIN [Altmetrics_2021Jun3].[dbo].[Journal] as j ON (alt.Altmetric_ID = j.Altmetric_ID)

  WHERE n.Posted_On >= '2014'
  AND (j.Journal IN ('arXiv', 'bioRxiv', 'ChemRxiv', 'medRxiv', 'Social Science Research Network (SSRN)')
  OR j.Altmetric_JID IN ('arxiv', 'biorxiv', 'chemrxiv', 'medrxiv', 'ssrn'))
```

In [10]:
input_file = 'data/news_of_preprints_mention_details.csv'
columns = ['Altmetric_ID', 'Order', 'DOI', 'Arxiv_ID', 'Journal', 'Altmetric_JID', 'First_Seen_On', 'PubDate', 'alt_title', 'Author_Name', 'Author_Url', 'Posted_On', 'Title', 'moreover_url']

In [11]:
df = pd.read_csv(input_file, sep=';', names=columns)
"{:,}".format(df.shape[0])

'173,802'

In [12]:
df.columns = [x.lower() for x in df.columns]
df.rename(columns = {'author_name': 'outlet', 'author_url': 'outlet_url'}, inplace=True)
df['posted_on'] = pd.to_datetime(df.posted_on)
df['posted_on_year'] = df.posted_on.map(lambda x: x.year)

In [13]:
df['server'] = df.apply(lambda row: row['altmetric_jid'] if pd.isna(row['journal']) else row['journal'], axis=1)
df['server'] = df.apply(lambda row: 'ssrn' if 'SSRN' in row['server'] else row['server'], axis=1)
df['server'] = df.server.str.lower()

# remove chemrxiv 
df = df[df.server != 'chemrxiv']
del df['journal']
del df['altmetric_jid']

df = df[df.outlet.isin(df2.index)]

In [14]:
# The last number might not be exactly right, because there are 2,413 some moreover_urls that are blank
"A total of {:,} mentions of {:,} preprints across {:,} news stories.".format(df.shape[0], df.altmetric_id.unique().shape[0], df[['outlet', 'moreover_url']].drop_duplicates().shape[0])

'A total of 40,039 mentions of 15,041 preprints across 31,258 news stories.'

In [15]:
df.to_csv('data/top outlets/preprints_mention_details.csv', index=False)